In [7]:
from pymongo import MongoClient
import os

print os.getcwd()

/Users/khaxis/workspace/image_tags/notebooks


In [2]:
client = MongoClient()
db = client['image_tags']

In [7]:
for row in db.image_urls.find({'NId':u'n00004475'}):
    print row['Url']

http://farm4.static.flickr.com/3175/2737866473_7958dc8760.jpg
http://farm4.static.flickr.com/3276/2875184020_9944005d0d.jpg
http://farm3.static.flickr.com/2531/4094333885_e8462a8338.jpg
http://farm4.static.flickr.com/3289/2809605169_8efe2b8f27.jpg
http://2.bp.blogspot.com/_SrRTF97Kbfo/SUqT9y-qTVI/AAAAAAAABmg/saRXhruwS6M/s400/bARADEI.jpg
http://fortunaweb.com.ar/wp-content/uploads/2009/10/Caroline-Atkinson-FMI.jpg
http://farm4.static.flickr.com/3488/4051378654_238ca94313.jpg
http://farm4.static.flickr.com/3368/3198142470_6eb0be5f32.jpg


ShowGraph
=========

In [46]:
def getDesctiption(nId):
    res = db.wordnet_words.find_one({'NId':nId})
    if res:
        return res['Description']
    return None

getDesctiption('n00017222')

u'plant, flora, plant life'

In [78]:
def findChildren(sourceNode, maxDepth=2):
    node = {'NId':sourceNode, 'children': list(), 'depth':0, 'description':getDesctiption(sourceNode)}
    res = node
    nodesToVisit = [node]
    while len(nodesToVisit) > 0:
        node = nodesToVisit[0]
        nodesToVisit = nodesToVisit[1:]
        if node['depth'] >= maxDepth:
            continue
        for row in db.wordnet_is_a.find({'NId':node['NId']}):
            newNode = {'NId':row['subNId'], 'children': list(), 'depth':node['depth']+1, 'description':getDesctiption(row['subNId'])}
            node['children'].append(newNode)
            nodesToVisit.append(newNode)
    return res
            
allChildren = findChildren('n00004475', 4)

In [82]:
def findParents(sourceNode):
    res = []
    node = {'NId':sourceNode, 'description':getDesctiption(sourceNode)}
    nodesToVisit = [node]
    while True:
        res.append(node)
        parent = db.wordnet_is_a.find_one({'subNId':node['NId']})
        if parent is None:
            break
        node = {'NId':parent['NId'], 'description':getDesctiption(parent['NId'])}
    return res

findParents(u'n00004475')

[{'NId': u'n00004475', 'description': u'organism, being'},
 {'NId': u'n00004258', 'description': u'living thing, animate thing'},
 {'NId': u'n00003553', 'description': u'whole, unit'},
 {'NId': u'n00002684', 'description': u'object, physical object'},
 {'NId': u'n00001930', 'description': u'physical entity'},
 {'NId': u'n00001740', 'description': u'entity'}]

In [80]:
def printTree(tree, depth=0):
    print '%s-%s: %s'%(' '*depth*4, tree['NId'], tree['description'])
    for child in tree['children']:
        printTree(child, depth+1)
printTree(allChildren)

-n00004475: organism, being
    -n00017222: plant, flora, plant life
        -n01383896: phytoplankton
            -n01401106: diatom
            -n01384084: planktonic algae
        -n11530008: microflora
        -n11530512: crop
            -n13086063: cash crop
            -n13086220: catch crop
            -n13086438: cover crop
            -n13086556: field crop
                -n12144399: field corn
            -n13126308: root crop
        -n11530715: endemic
        -n11530860: holophyte
        -n11530990: non-flowering plant
        -n11531090: plantlet
        -n11531193: wilding
            -n11672400: wildflower, wild flower
                -n11721844: sagebrush buttercup, Ranunculus glaberrimus
                -n11737534: pasqueflower, pasque flower
                -n11738547: meadow rue
                -n11836722: sand verbena
                -n11838413: trailing four o'clock, trailing windmills, Allionia incarnata
                -n11858814: red maids, redmaids, Calandr

In [85]:
def getNIdsRecursive(treeNode, res):
    res.append(treeNode['NId'])
    for child in treeNode['children']:
        getNIdsRecursive(child, res)

def getTreeUrls(tree):
    res = []
    nIds = []
    getNIdsRecursive(tree, nIds)
    
    for row in db.image_urls_collection.find({'$or': [{'NId':nid} for nid in nIds]}):
        res.append(row['Urls'])
    
    return res

len(getTreeUrls(allChildren))

2704

In [86]:
allChildren['description']

u'organism, being'

In [6]:
import requests

def downloadSingleImage(url, dest):
	#urllib.urlretrieve (url, dest)
	# Make the actual request, set the timeout for no data to 10 seconds and enable streaming responses so we don't have to keep the large files in memory
	try:
		request = requests.get(url, timeout=1, stream=True, allow_redirects=False)
		print request.status_code
		if request.status_code == 301:
			return False    
	except:
		return False
	# Open the output file and make sure we write in binary mode
	with open(dest, 'wb') as fh:
		# Walk through the request response in chunks of 1024 * 1024 bytes, so 1MiB
		try:
			for chunk in request.iter_content(1024 * 1024):
				# Write the chunk to the file
				fh.write(chunk)
				# Optionally we can check here if the download is taking too long
		except:
			return False
	return True

In [7]:
url = 'http://articles.mercola.com/ImageServer/Public/2006/08-august/8.14gut.jpg'
#url = 'http://floridakiss.com/kiss_e_gram/April08/images/aquaticaPlunge.jpg'
url = 'http://floridakiss.com/kiss_e_gram/April08/images/aquaticaPlunge.jpg'
downloadSingleImage(url, '/tmp/a.jpg')

301


False

In [4]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # dirty hack, but ok for research
from utils import img_collection as icoll